# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846820955855                   -0.70    4.5
  2   -7.852304380443       -2.26       -1.53    1.0
  3   -7.852646201267       -3.47       -2.52    3.2
  4   -7.852646677456       -6.32       -3.37    2.2
  5   -7.852646686210       -8.06       -4.66    1.2
  6   -7.852646686728       -9.29       -5.49    3.2
  7   -7.852646686730      -11.67       -6.26    1.2
  8   -7.852646686730      -12.98       -7.64    2.0
  9   -7.852646686730      -14.75       -7.84    3.2
 10   -7.852646686730   +    -Inf       -8.56    2.0
 11   -7.852646686730      -14.75       -9.11    1.8
 12   -7.852646686730   +    -Inf      -10.28    2.0
 13   -7.852646686730   +    -Inf      -11.00    3.2
 14   -7.852646686730   +  -15.05      -11.56    2.2
 15   -7.852646686730      -15.05      -12.22    1.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846813372393                   -0.70           4.5
  2   -7.852527194228       -2.24       -1.63   0.80    2.0
  3   -7.852637245772       -3.96       -2.71   0.80    1.0
  4   -7.852646471344       -5.04       -3.31   0.80    2.2
  5   -7.852646675668       -6.69       -4.15   0.80    1.2
  6   -7.852646686427       -7.97       -4.75   0.80    1.5
  7   -7.852646686712       -9.55       -5.57   0.80    1.5
  8   -7.852646686730      -10.74       -6.41   0.80    2.2
  9   -7.852646686730      -12.65       -7.60   0.80    2.2
 10   -7.852646686730      -14.27       -8.50   0.80    2.8
 11   -7.852646686730   +  -14.75       -9.20   0.80    2.0
 12   -7.852646686730      -14.35      -10.55   0.80    2.0
 13   -7.852646686730   +  -14.57      -10.86   0.80    3.2
 14   -7.852646686730   +    -Inf      -12.19   0.80    1.0


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.375037e+01     3.101447e+00
 * time: 0.46505022048950195
     1     1.252097e+00     1.763548e+00
 * time: 0.712468147277832
     2    -1.181582e+00     1.899992e+00
 * time: 0.7395310401916504
     3    -3.650181e+00     1.614201e+00
 * time: 0.779583215713501
     4    -4.885063e+00     1.574691e+00
 * time: 0.8195371627807617
     5    -6.730795e+00     1.098430e+00
 * time: 0.858896017074585
     6    -7.406603e+00     5.270824e-01
 * time: 0.9012081623077393
     7    -7.681515e+00     2.342594e-01
 * time: 0.9291541576385498
     8    -7.778887e+00     9.371923e-02
 * time: 0.9558370113372803
     9    -7.809376e+00     1.234816e-01
 * time: 0.9854550361633301
    10    -7.829767e+00     5.671771e-02
 * time: 1.0133981704711914
    11    -7.840804e+00     7.089718e-02
 * time: 1.0408101081848145
    12    -7.844359e+00     8.158196e-02
 * time: 1.0683701038360596
    13    -7.846083e+00     6.278191e-02
 * time: 1.095793008804

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846853874971                   -0.70    4.8
  2   -7.852319297897       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686704                   -2.55
  2   -7.852646686730      -10.59       -5.96
  3   -7.852646686730   +  -14.45      -12.70


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.1131167501702985e-13
|ρ_newton - ρ_scfv| = 1.7915193087580252e-12
|ρ_newton - ρ_dm|   = 2.5622673046735726e-9
